In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tifffile
import h5py
import os
import glob
from PIL import Image
from skimage.transform import resize
from tqdm import tqdm
from shutil import copyfile

import tifffile

In [2]:
%pip install yacs
from yacs.config import CfgNode as CN

Note: you may need to restart the kernel to use updated packages.


In [23]:
_C = CN()

#_C.H5IMGS = '/n/pfister_lab2/Lab/zudilin/data/Expansion/image/dorsal_6_histeq.h5'
#_C.H5IMGS = '/n/pfister_lab2/Lab/leander/cerberus/ccgan/datasets/dorsal_crop_3D_255_512/testB/exm_3D_255_512_512.tif'
_C.H5IMGS = '/n/pfister_lab2/Lab/zudilin/data/NucExM/NucExM-Release/downsampled/image4_ds_4x.h5'


#_C.SAVEB = '/n/pfister_lab2/Lab/leander/em2exm/ccgan/datasets/dorsal_crop_volume/trainB/'#'../../cgan/pytorch-CycleGAN-and-pix2pix/datasets/em2exm/sub_dataset_round/testB/F000_Round1_c4_' 
#_C.SAVEB = '/n/pfister_lab2/Lab/leander/em2exm/ccgan/datasets/submission_255_512_512/testB'#'../../cgan/pytorch-CycleGAN-and-pix2pix/datasets/em2exm/sub_dataset_round/testB/F000_Round1_c4_' 
_C.SAVEB = '/n/pfister_lab2/Lab/leander/em2exm/ccgan/datasets/submission_vol4_255_512/trainB/'
#_C.SAVEB = '/n/pfister_lab2/Lab/leander/em2exm/ccgan/datasets/submission_vol4_255_512/testB/'

_C.BLOCKSIZE = [128,128]#[2048,2048]
#_C.SCALE = [0.25,0.25] #[1/16,1/16] 
_C.SCALE = [1,1]

_C.NRSAVEIMG = float("inf") # number of image blocks that should be saved

_C.FILTER = False

In [24]:
def get_cfg_defaults():
  """Get a yacs CfgNode object with default values for my_project."""
  # Return a clone so that the defaults will not be altered
  # This is for the "local variable" use pattern
  return _C.clone()

In [25]:
cfg = get_cfg_defaults()
#cfg.merge_from_file("./configs/base_setup.yaml")
cfg.freeze()

In [26]:
def blockshaped(arr, height, width, pad=False):
    """
    Resizes the 2D matrix arr to a mutlipte of height and width to be devidable with out rest.
    The default is to crop the matrix. If pad is set to true the matrix is extended with zeros.
    """
    h, w = arr.shape
    
    assert h >= height, f"Image height {h} must be larger equal {height}"
    assert w >= width, f"Image width {w} must be larger equal {width}"

    if not pad:
        arr_resized = arr[:(height*int(h/height)), :(width*int(w/width))]
        h, w = arr_resized.shape
    else:
        h_pad = (height - (h% height))
        w_pad = (width - (w% width))
        arr_resized = np.zeros((h+h_pad, w+w_pad))
        arr_resized[:h, :w] = arr

    assert h % height == 0, f"{h} rows is not evenly divisible by {height}"
    assert w % width == 0, f"{w} cols is not evenly divisible by {width}"
    return int(h/height), int(w/width), (arr_resized.reshape(h//height, height, -1, width)
               .swapaxes(1,2)
               .reshape(-1, height, width))


In [27]:
def create_h5s(img_list, img_file_name, size=None):
    """ Take a list of images and writes them to a h5 file
    
        Args:
            img_list: List of images that should be written to the h5-file
            img_file_name: Name for the h5-file
            size: Size to which the images are resized, has to be an int tuple
    """

    
    # open a h5-file for writing
    with h5py.File(img_file_name, "w") as img_patches:
        
        # create the dataset specifing key and shape of the dataset
        if size is not None:
            assert isinstance(size, tuple)
            img_patches = img_patches.create_dataset('main',shape=(len(img_list),size[0],size[1]), dtype=int)
        else:
            img_patches = img_patches.create_dataset('main',shape=(len(img_list),img_list[0].shape[0],img_list[0].shape[1]), dtype=int)
        
        # iterate over all images in the list
        for i, img in tqdm(enumerate(img_list)):

            # resize the images
            if size is not None: 
                img = cv2.resize(img, (size[1],size[0]))

            # add the resized image to the dataset
            img_patches[i:i+1,:,:] = img

# Extracting sub-volume from dorsal 6

In [28]:
UPSAMPLE = cfg.SCALE
BLOCK_SIZE = cfg.BLOCKSIZE

with h5py.File(cfg.H5IMGS) as hdf_dorsal:
#with tifffile.TiffFile(cfg.H5IMGS) as hdf_dorsal:
    
    # get the masks
    hdf_dorsal_keys = list(hdf_dorsal.keys()) # prints ['main']
    hdf_dors = hdf_dorsal.get(hdf_dorsal_keys[0])
    #hdf_dors = np.asarray([e.asarray() for e in hdf_dorsal.pages[:]])
    print(hdf_dors[0].shape)
    
    IMG_SIZE_H = hdf_dors[0,:,:].shape[0] *  UPSAMPLE[0]
    IMG_SIZE_W = hdf_dors[0,:,:].shape[1] *  UPSAMPLE[1]
    
    IMG_SIZE_H = hdf_dors[0,:,:].shape[0] * UPSAMPLE[0]
    IMG_SIZE_W = hdf_dors[0,:,:].shape[1] * UPSAMPLE[1]

    max_pixel = np.array(hdf_dors).max()
    min_pixel = np.array(hdf_dors).min()
    std = np.array(hdf_dors).std()
    mean = np.mean(np.array(hdf_dors))
    
    print("MAX: ", max_pixel)
    print("MIN: ", min_pixel)
        
    print("STD: ", std)
    print("MEAN: ", mean)

    c_img = 0
    c_patch =0
    
    # loop over all images in a dataset
    for i, h in tqdm(enumerate(hdf_dors)):

        # normalize based on max and min of current dataset
        h = (h-min_pixel)/(max_pixel-min_pixel)
        # resize the image
        h = resize(h, (IMG_SIZE_H, IMG_SIZE_W))

        # mean of current image
        h_mean = np.mean(h)

        # cut in to blocks
        rows, cols, img_blocks = blockshaped(h,BLOCK_SIZE[0],BLOCK_SIZE[1])

        # loop over the blocks
        for j in range (0, img_blocks.shape[0]): 
            # only save those blocks whose mean are above the mean of the whole image
            if not cfg.FILTER:
                # save the image block
                Image.fromarray((img_blocks[j,:,:] * 255).astype('uint8'), mode='L').save(os.path.join(cfg.SAVEB,str(i)+'_block_'+str(j)+'.png'))                
                c_patch +=1
                if c_patch == cfg.NRSAVEIMG:
                    break
            if cfg.FILTER and np.mean(img_blocks[j,:,:]) >= h_mean:
                # save the image block
                Image.fromarray((img_blocks[j,:,:] * 255).astype('uint8'), mode='L').save(os.path.join(cfg.SAVEB,str(i)+'_block_'+str(j)+'.png'))                
                c_patch +=1
                if c_patch == cfg.NRSAVEIMG:
                    break
        if c_patch == cfg.NRSAVEIMG:
            break
        c_img +=1

    
print(c_img,c_patch)

/n/home05/lauenburg/.conda/envs/img_toolbox/lib/python3.7/site-packages/ipykernel_launcher.py:4: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  after removing the cwd from sys.path.


(512, 512)
MAX:  255
MIN:  0
STD:  47.924342929083856
MEAN:  71.76749828563017


255it [05:12,  1.22s/it]

255 4080


## Filter for specific slices and patches

In [29]:
#filter for sclices
def filter_for_slice_range(slice_range):
    def filter_slice(name):
        for nr in slice_range:
            if int(name.split('/')[-1].split('_')[0]) == nr:
                return True
        return False
    return filter_slice

# filter for block
def filter_for_block_set(block_set):
    def filter_block(name):
        for nr in block_set:
            if int(name.split('/')[-1].split('_')[-1].split('.')[0]) == nr:
                return True
        return False
    return filter_block

In [30]:
blocks = glob.glob(cfg.SAVEB+'*')
slice_range = list(range(75,175))
filter_sl_range = filter_for_slice_range(slice_range)
filtered_blocks_range = list(filter(filter_sl_range, blocks))
block_set = [5,6,9,10]
filter_bl = filter_for_block_set(block_set)
filtered_block_set = list(filter(filter_bl, filtered_blocks_range))
print(len(filtered_block_set))
print(filtered_block_set)

400
['/n/pfister_lab2/Lab/leander/em2exm/ccgan/datasets/submission_vol4_255_512/trainB/75_block_5.png', '/n/pfister_lab2/Lab/leander/em2exm/ccgan/datasets/submission_vol4_255_512/trainB/75_block_6.png', '/n/pfister_lab2/Lab/leander/em2exm/ccgan/datasets/submission_vol4_255_512/trainB/75_block_9.png', '/n/pfister_lab2/Lab/leander/em2exm/ccgan/datasets/submission_vol4_255_512/trainB/75_block_10.png', '/n/pfister_lab2/Lab/leander/em2exm/ccgan/datasets/submission_vol4_255_512/trainB/76_block_5.png', '/n/pfister_lab2/Lab/leander/em2exm/ccgan/datasets/submission_vol4_255_512/trainB/76_block_6.png', '/n/pfister_lab2/Lab/leander/em2exm/ccgan/datasets/submission_vol4_255_512/trainB/76_block_9.png', '/n/pfister_lab2/Lab/leander/em2exm/ccgan/datasets/submission_vol4_255_512/trainB/76_block_10.png', '/n/pfister_lab2/Lab/leander/em2exm/ccgan/datasets/submission_vol4_255_512/trainB/77_block_5.png', '/n/pfister_lab2/Lab/leander/em2exm/ccgan/datasets/submission_vol4_255_512/trainB/77_block_6.png', '/n

In [ ]:
# move the selected slices to the trainB folder
for img in filtered_block_set:
    copyfile(img, os.path.join('/n/pfister_lab2/Lab/leander/cgan/pytorch-CycleGAN-and-pix2pix/results/256_not_filtered_dorsal_crop/trainB', img.split('/')[-1]))

In [ ]:
# exspects 2x2 image patches per slice
from skimage.transform import resize

with h5py.File('/n/pfister_lab2/Lab/leander/em2exm/pytorch_connectomics/datasets/inference/merged_256_not_filtered_dorsal_b.h5') as images:    
    imgs = images.get('main')
    sclice_count = [i*4 for i in range(0,100)]
    merged_img_patches = []
    for i in sclice_count[:]:
        # Merge image patches
        patches_top = np.concatenate((imgs[i],imgs[i+1]),axis=1)
        patches_bot = np.concatenate((imgs[i+2],imgs[i+3]),axis=1)
        merged_img_patches.append(np.concatenate((patches_top,patches_bot),axis=0))

#plt.imshow(patch_work)
merged_img_patches = np.asarray(merged_img_patches)
print(merged_img_patches.shape)


create_h5s(merged_seg_patches,"./dense_seg_20E_15DE_3SW_SH_merged_248.h5", size=(256,256))
create_h5s(merged_img_patches,"./dense_img_20E_15DE_3SW_SH_merged_248.h5", size=(256,256))
        